In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import gzip
import fastparquet
import os 
from datetime import datetime, timedelta 
from tqdm import tqdm, trange

In [2]:
#on commence par lire la liste de sstations, on utilise 
#alors cette liste de stations pour déterminer les datasets à lire et ainsi on lit les div
data_directory = '/media/parisnic/STOCKAGE/M2/GNSS_US/'

meta = pd.read_csv('stations.csv')
stations = meta['name'].values
lats = meta['lat'].values
lons = meta['lon'].values
heights = meta['height'].values
print(stations)

#maintenant on va chercher les fichiers pour 
year = 2023
dataframes_dict = {}

# Loop through the file names and load dataframes into the dictionary
for sta in stations:
    # Load the dataframe from the parquet file
    dataframe = pd.read_parquet(f'{data_directory}{sta}/data_{year}.parquet')
    dataframes_dict[sta] = dataframe

#on cherche la valeur de vmax (et vmin comme étant - vmax  ) pour tous les types de données pour l'ensemble des stations et pour n'importe quel temps -> permet après
#d'avoir un cmap fixe au cours du temps

vmax_vref = 0
vmax_nref = 0
vmax_eref = 0
vmax_vmean = 0
vmax_nmean = 0
vmax_emean = 0

for i,sta in enumerate(stations):
    temp_vref = np.max(np.abs(dataframes_dict[sta]['v-ref(m)'].values))
    temp_nref = np.max(np.abs(dataframes_dict[sta]['n-ref(m)'].values))
    temp_eref = np.max(np.abs(dataframes_dict[sta]['e-ref(m)'].values))
    temp_vmean = np.max(np.abs(dataframes_dict[sta]['v-mean(m)'].values))
    temp_nmean = np.max(np.abs(dataframes_dict[sta]['n-mean(m)'].values))
    temp_emean = np.max(np.abs(dataframes_dict[sta]['e-mean(m)'].values))
    
    if temp_vref>vmax_vref and temp_vref<1:
        vmax_vref=temp_vref
    if temp_nref>vmax_nref and temp_nref<1:
        vmax_nref=temp_nref
    if temp_eref>vmax_eref and temp_eref<1:
        vmax_eref=temp_eref
    if temp_vmean>vmax_vmean and temp_vmean<1:
        vmax_vmean=temp_vmean
    if temp_nmean>vmax_nmean and temp_nmean<1:
        vmax_nmean=temp_nmean
    if temp_emean>vmax_emean and temp_emean<1:
        vmax_emean=temp_emean

print(vmax_vref,vmax_nref,vmax_eref,vmax_vmean,vmax_nmean,vmax_emean)

['CAM6' 'MJPK' 'P471' 'SACY' 'SIO5' 'TRAK' 'VTIS' 'LBC1' 'PVHS' 'MHMS'
 'BKMS' 'FVPK' 'CACL' 'P474' 'LBNO' 'P472' 'P475' 'CACJ' 'RAAP' 'CSDH'
 'P470' 'HOL3' 'RSTP' 'P583']
0.57484 0.562835 0.928909 0.547306 0.223273 0.592646


In [3]:
#maintenant qu'on a chargé les dataframes, on va pouvoir itérer sur les dataframes au sein d'un itération sur le temps
#on créé à présent l'array de temps sur lequel on va chercher les valeurs aux différentes stations

path2export = '/media/parisnic/STOCKAGE/M2/maps/'

tbeg = datetime(2023,8,27)
tend = datetime(2023,8,28)
times = [tbeg]
while times[-1]!=tend:
    times.append(times[-1]+timedelta(minutes=5))

    
for i,t in tqdm(enumerate(times)):
    n_refs  = np.zeros(len(stations))
    e_refs  = np.zeros(len(stations))
    v_refs  = np.zeros(len(stations))
    
    n_means  = np.zeros(len(stations))
    e_means  = np.zeros(len(stations))
    v_means  = np.zeros(len(stations))
    for j, sta in enumerate(stations):
        try:
            idx = np.where(dataframes_dict[sta]['Time_UTC']==t)[0][0] #trouve l'indice de dans le tableau correspondant au bon temps -> on va alors chercher les params à cetindice pour cette station
            n_refs[j] = dataframes_dict[sta]['n-ref(m)'].values[idx]
            e_refs[j] = dataframes_dict[sta]['e-ref(m)'].values[idx]
            v_refs[j] = dataframes_dict[sta]['v-ref(m)'].values[idx]
            n_means[j] = dataframes_dict[sta]['n-mean(m)'].values[idx]
            e_means[j] = dataframes_dict[sta]['e-mean(m)'].values[idx]
            v_means[j] = dataframes_dict[sta]['v-mean(m)'].values[idx]
        except:
            n_refs[j] = np.nan
            e_refs[j] = np.nan
            v_refs[j] = np.nan
            n_means[j] = np.nan
            e_means[j] = np.nan
            v_means[j] = np.nan
    
    #maintenant qu'on a les valeurs des différentes station au temps considéré, il nous faut à présent plotter la carte, on commence par la faire en 2D sans altitute
    fig, ax = plt.subplots(2,3, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.5)
    im0 = ax[0,0].scatter(lons,lats,c=v_refs,cmap='bwr',vmax=vmax_vref,vmin=-vmax_vref); ax[0,0].set_title('v-ref (m)');plt.colorbar(im0, ax=ax[0, 0])
    im1 = ax[0,1].scatter(lons,lats,c=n_refs,cmap='bwr',vmax=vmax_nref,vmin=-vmax_nref); ax[0,1].set_title('n-ref (m)');plt.colorbar(im1, ax=ax[0, 1])
    im2 = ax[0,2].scatter(lons,lats,c=e_refs,cmap='bwr',vmax=vmax_eref,vmin=-vmax_eref); ax[0,2].set_title('e-ref (m)');plt.colorbar(im2, ax=ax[0, 2])
    im3 = ax[1,0].scatter(lons,lats,c=v_means,cmap='bwr',vmax=vmax_vmean,vmin=-vmax_vmean); ax[1,0].set_title('v-mean (m)');plt.colorbar(im3, ax=ax[1, 0])
    im4 = ax[1,1].scatter(lons,lats,c=n_means,cmap='bwr',vmax=vmax_nmean,vmin=-vmax_nmean); ax[1,1].set_title('n-mean (m)');plt.colorbar(im4, ax=ax[1, 1])
    im5 = ax[1,2].scatter(lons,lats,c=e_means,cmap='bwr',vmax=vmax_emean,vmin=-vmax_emean); ax[1,2].set_title('e-mean (m)');plt.colorbar(im5, ax=ax[1, 2])
    plt.savefig(f'{path2export}{t}.png',bbox_inches='tight')
    
    plt.close('all')
    
    
# print(dataframes_dict['CAM6']['Time_UTC'][0])

289it [05:30,  1.14s/it]
